# load

In [1]:
import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F



class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model
model1 = CNNPointDetector()
# 初始化模型
model1 = CNNPointDetector()
model1.load_state_dict(torch.load('fan_corner1_detector_1201.pth'))  # 加载模型参数
model1.train()

C:\Users\user\AppData\Local\Temp\ipykernel_18828\3844279047.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load('fan_corner1_detector_1201

CNNPointDetector(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

# newstart

In [4]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points1_1127.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model1
model1 = CNNPointDetector()

In [6]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

# Training loop
num_epochs = 720
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model1(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about',(num_epochs-epoch)/1,'min   or %-.3f hrs left'% ((num_epochs-epoch)/60))

Epoch [1/720], Loss: 5189451.2143 about 720.0 min   or 12.000 hrs left
Epoch [2/720], Loss: 5181998.8571 about 719.0 min   or 11.983 hrs left
Epoch [3/720], Loss: 5032874.7143 about 718.0 min   or 11.967 hrs left
Epoch [4/720], Loss: 4487036.5357 about 717.0 min   or 11.950 hrs left
Epoch [5/720], Loss: 3046050.8214 about 716.0 min   or 11.933 hrs left
Epoch [6/720], Loss: 844520.3951 about 715.0 min   or 11.917 hrs left
Epoch [7/720], Loss: 542680.6429 about 714.0 min   or 11.900 hrs left
Epoch [8/720], Loss: 256872.4888 about 713.0 min   or 11.883 hrs left
Epoch [9/720], Loss: 311041.4598 about 712.0 min   or 11.867 hrs left
Epoch [10/720], Loss: 212476.4219 about 711.0 min   or 11.850 hrs left
Epoch [11/720], Loss: 211892.8237 about 710.0 min   or 11.833 hrs left
Epoch [12/720], Loss: 197458.0089 about 709.0 min   or 11.817 hrs left
Epoch [13/720], Loss: 196552.6741 about 708.0 min   or 11.800 hrs left
Epoch [14/720], Loss: 190728.0536 about 707.0 min   or 11.783 hrs left
Epoch [15/

In [7]:
torch.save(model1.state_dict(), './fan_corner1_detector_1129.pth')

# test

In [2]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points1_1126test.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
def evaluate(model, dataset, num_samples=30):
    model.eval()
    with torch.no_grad():
        for i in range(num_samples):
            img, label = dataset[i]
            img = img.unsqueeze(0)  # Add batch dimension
            
            # Model prediction
            prediction = model(img)
            print(f"True: {label.numpy()}, Predicted: {prediction.numpy().flatten()}")

# Test on some sample
evaluate(model1, dataset)

True: [2374. 2429.], Predicted: [2423.295  2441.0623]
True: [2869. 2437.], Predicted: [2928.2515 2430.4548]
True: [2907. 2384.], Predicted: [2916.6343 2436.406 ]
True: [2924. 2309.], Predicted: [2758.9136 2373.6155]
True: [2662. 1697.], Predicted: [2717.3665 1648.6547]
True: [2543. 1531.], Predicted: [2599.216  1473.0841]
True: [2411. 1355.], Predicted: [2387.5046 1399.9906]
True: [2469. 1651.], Predicted: [2471.9438 1716.7026]
True: [2491. 1765.], Predicted: [2494.1821 1788.6759]
True: [2492. 1862.], Predicted: [2508.075  1861.0913]
True: [2219. 2124.], Predicted: [2187.7676 2182.4226]
True: [2112. 2129.], Predicted: [2189.7058 2101.367 ]
True: [2422. 2634.], Predicted: [2426.7383 2582.4756]
True: [1992. 2117.], Predicted: [2013.2476 2103.5847]
True: [1164. 1824.], Predicted: [1162.9723 1801.9717]
True: [1252. 1934.], Predicted: [1240.7291 1958.0422]
True: [1394. 2077.], Predicted: [1370.2961 2025.2651]
True: [2496. 2736.], Predicted: [2577.8591 2752.4128]
True: [1943. 2556.], Predict

# keep training

In [3]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points1_1127.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

# Training loop
num_epochs = 1200
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model1(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about',(num_epochs-epoch)/1,'min   or %-.3f hrs left'% ((num_epochs-epoch)/60))

Epoch [1/1200], Loss: 15852.4753 about 1200.0 min   or 20.000 hrs left
Epoch [2/1200], Loss: 6483.5093 about 1199.0 min   or 19.983 hrs left
Epoch [3/1200], Loss: 3618.6991 about 1198.0 min   or 19.967 hrs left


In [ ]:
torch.save(model1.state_dict(), './fan_corner1_detector_1202.pth')